## ETL Pipeline Preparation

### 1. Import libraries and load datasets.
Import Python libraries
Load messages.csv into a dataframe and inspect the first few lines.
Load categories.csv into a dataframe and inspect the first few lines.

In [ ]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

pd.set_option('max_rows', 1000)
pd.set_option('max_columns', 1000)

In [ ]:
# load messages dataset
messages = pd.read_csv('messages.csv')
messages.head()

In [ ]:
# load categories dataset
categories = pd.read_csv('categories.csv')
categories.head()

### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [ ]:
# merge datasets
df = pd.merge(messages, categories, on = 'id')
df.head()

### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. Set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [ ]:
# create a dataframe of the 36 individual category columns
categories = pd.DataFrame(categories['categories'].str.split(';', expand = True))
categories.head()

In [ ]:
# select the first row of the categories dataframe
col_names = categories.iloc[0]

# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
col_names = [x[:-2] for x in col_names]
print(col_names)

In [ ]:
# rename the columns of `categories`
categories.columns = col_names
categories.head()

### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- Perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. 

In [ ]:
for column in categories:
    # set each value to be the last character of the string
    categories[column] = categories[column].str[-1:]
    
    # convert column from string to numeric
    categories[column] = categories[column].astype(float)
categories.head()

### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [ ]:
# drop the original categories column from `df`
df = df.drop(['categories'], axis = 1)

df.head()

In [ ]:
# concatenate the original dataframe with the new `categories` dataframe
# joining on index as the two dataframes have no common columns
df = df.join(categories, how='outer')
df.head()

### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [ ]:
# check number of duplicates
duplicates = df['id'].loc[df['id'].duplicated().sum()]

print(duplicates)

In [ ]:
# drop duplicates
df = df.drop_duplicates(subset = ['id'],keep = 'first')
#df.head()

In [ ]:
# check number of duplicates
duplicates = df['id'].nunique()

print(duplicates)

print(df.shape)

### 7. Save the clean dataset into an sqlite database.
Use the pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library.

In [ ]:
engine = create_engine('sqlite:///processed_etl_pipeline_data.db')
df.to_sql('processed_etl_pipeline_data', engine, index=False)